<a href="https://colab.research.google.com/github/hadi-M/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module1/HADI_MODARES_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np
from scipy import stats
from pdb import set_trace as st

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

In [5]:
df = pd.read_csv('house-votes-84.data', names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [11]:
df1 = df.replace({"?": np.NaN, "y": 1, "n": 0})
df1.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
def column_difference_significance_checker(df, column_name, critical_value):
  rep = df[df['party']=='republican'][column_name]
  dem = df[df['party']=='democrat'][column_name]
  # print(stats.ttest_ind(dem, rep, nan_policy="omit"))
  p_value = stats.ttest_ind(rep, dem, nan_policy="omit")[1]
  if p_value < critical_value:
    if dem.mean() > rep.mean():
      return "democrats", p_value
    else:
      return "republicans", p_value
  else:
    return -1, p_value


def dataframe_difference_significance_checker(df, critical_value):
  bill_columns = list(df.columns)[1:]
  significant_columns = {
      "column_name": [],
      "supporter": [],
      "p_value": []
  }
  insignificant_columns = {
      "column_name": [],
      "p_value": []
  }

  for column in bill_columns:
    res = column_difference_significance_checker(df, column, critical_value)
    
    if res[0] == -1:
      insignificant_columns["column_name"].append(column)
      insignificant_columns["p_value"].append(res[1])

    else:
      significant_columns["column_name"].append(column)
      significant_columns["supporter"].append(res[0])
      significant_columns["p_value"].append(res[1])

  significant_df = pd.DataFrame(significant_columns).sort_values("supporter")
  insignificant_df = pd.DataFrame(insignificant_columns)

  return significant_df, insignificant_df

significant_df, insignificant_df = dataframe_difference_significance_checker(df1, 0.01)

In [36]:
significant_df

,column_name,supporter,p_value
0,handicapped-infants,democrats,1.613440e-18
1,budget,democrats,2.070340e-77
5,anti-satellite-ban,democrats,8.521033e-31
6,aid-to-contras,democrats,2.824718e-54
7,mx-missile,democrats,5.030793e-47
8,synfuels,democrats,1.575932e-15
12,duty-free,democrats,5.997697e-32
13,south-africa,democrats,3.652674e-11
2,physician-fee-freeze,republicans,1.994262e-177
3,el-salvador-aid,republicans,5.600520e-68


In [37]:
insignificant_df

,column_name,p_value
0,water-project,0.929156
1,immigration,0.083302


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!